### Latex macros:
$$
\newcommand{\E}{\text{E}}
\newcommand{\mbf}{\mathbf}
\newcommand{\bs}{\boldsymbol}
\newcommand{\Var}{\text{Var}}
\newcommand{\Cov}{\text{Cov}}
\newcommand{\e}{\frac{1}{\sigma^2_e}}
\newcommand{\f}{\frac{1}{\sigma^2_{\alpha}}}
$$

The formula
$$
\mbf{A}^{-1} = 
   \begin{bmatrix}
    \mbf{A}_{11}^{-1} & \mbf{0} \\
    \mbf{0}      & \mbf{0}
    \end{bmatrix}
+
   \begin{bmatrix}
    \mbf{q}\mbf{q}' & -\mbf{q} \\
    -\mbf{q}'       & 1
    \end{bmatrix}\frac{1}{r}
$$

can also be used to compute $\mbf{A}^{-1}$ iteratively by starting with $\mbf{A}_{11} = a_{11}$ and expanding it one row and column at a time.

Consider, for example, the matrix:
$$
\mathbf{A} = 
   \begin{bmatrix}
    2.0 & 0.5 & 0.2 \\
    0.5 & 3.0 & 0.1 \\
    0.2 & 0.1 & 4.0
\end{bmatrix}
$$

In [11]:
A = [2.0 0.5 0.2
     0.5 3.0 0.1
     0.2 0.1 4.0]

3×3 Array{Float64,2}:
 2.0  0.5  0.2
 0.5  3.0  0.1
 0.2  0.1  4.0

In [3]:
# Inverse of upper 1x1 matrix
A11 = A[1,1]
A11i = 1/A11

0.5

In [4]:
# Inverse of upper 2x2 matrix
A22 = A[2,2]
A12 = A[1,2]

q = A11i*A12
r = A22 - A12'A11i*A12
ri = 1/r
Ai = [A11i .+ ri*q*q'    -q*ri
      -q'*ri              ri]

2×2 Array{Float64,2}:
  0.521739   -0.0869565
 -0.0869565   0.347826 

### Check with Julia inverse function  

In [5]:
s=1:2
inv(A[s,s])

2×2 Array{Float64,2}:
  0.521739   -0.0869565
 -0.0869565   0.347826 

In [6]:
# Inverse of upper 3x3 matrix
s1 = 1:2
s2 = 3
A22 = A[s2,s2]
A12 = A[s1,s2]
A11i = Ai

q = A11i*A12
r = A22 - A12'A11i*A12
ri = 1/r
Ai = [A11i .+ ri*q*q'    -q*ri
      -q'*ri              ri]

3×3 Array{Float64,2}:
  0.524038   -0.0865385   -0.0240385 
 -0.0865385   0.347902    -0.00437063
 -0.0240385  -0.00437063   0.251311  

### Check with Julia inverse function  

In [7]:
s=1:3
inv(A[s,s])

3×3 Array{Float64,2}:
  0.524038   -0.0865385   -0.0240385 
 -0.0865385   0.347902    -0.00437063
 -0.0240385  -0.00437063   0.251311  

## Exercise

Write an iterative function in Julia to invert a symmetric positive definite matrix. Compare the speed of this function with that of the recursive function. 

In [1]:
n = 1000
p = 1000
M=rand(n,p)
A=M'M;

In [2]:
function invA(A)
    n  = size(A,1)
    if n==1
        return 1/A[1,1]
    else
        Ai = zeros(n,n) # inverse of A
        s1 = 1:n-1
        s2 = n
        A11 = A[s1,s1]
        A11i = invA(A11)
        A12 = A[s1,s2]
        A22 = A[s2,s2]
        q = A11i*A12
        r = A22 - q'A11*q
        ri = 1/r
        Ai = [A11i .+ ri*q*q'    -q*ri
              -q'*ri              ri]
        return Ai
   end
end

invA (generic function with 1 method)

In [3]:
function invAiter(A)
    n = size(A,1)
    Ai = 1.0/A[1,1]
    for i=2:n
        A11i = Ai   
        s1 = 1:i-1
        s2 = i 
        A11 = A[s1,s1]   
        A12 = A[s1,s2]
        A22 = A[s2,s2]
        L = A11i*A12
        R = A22 - L'A11*L
        Ri = 1/R[1,1]
        Ai = [A11i  .+ (Ri.* L*L')  -Ri.*L
              -Ri.*L'                Ri.*1]
    end
    return Ai
end     

invAiter (generic function with 1 method)

In [4]:
# complete function
function invAME(A)
    n = size(A,1)
    Ai = zeros(n,n)
    Ai[1,1] = 1.0/A[1,1]
    for i=2:n   
        s1 = 1:i-1
        s2 = i 
        A12 = A[s1,s2]
        A22 = A[s2,s2]
        L = Ai[s1,s1]*A12
        R = A22 - A12'Ai[s1,s1]*A12
        Ri = 1/R[1,1]
        Ai[s1,s1] += (Ri.* L*L')
        Ai[s1,s2] = -Ri.*L
        Ai[s2,s1] = -Ri.*L' 
        Ai[s2,s2] =  Ri.*1
    end
    return Ai
end                    

invAME (generic function with 1 method)

In [43]:
# complete function
function invAME1(A)
    n = size(A,1)
    Ai = zeros(n,n)
    Ai[1,1] = 1.0/A[1,1]
    for i=2:n   
        s1 = 1:i-1
        s2 = i 
        A12 = A[s1,s2]
        A22 = A[s2,s2]
        L = Ai[s1,s1]*A12
        R = A22 - A12'Ai[s1,s1]*A12
        Ri = 1/R[1,1]
        for j in s1, k in s1
            Ai[j,k] += Ri*L[j]*L[k]
        end    
        Ai[s1,s2]  = -Ri.*L
        Ai[s2,s1]  = -Ri.*L 
        Ai[s2,s2]  =  Ri.*1
    end
    return Ai
end                    

invAME1 (generic function with 1 method)

In [9]:
@time invA(A);

 58.241530 seconds (81.59 k allocations: 14.935 GiB, 40.95% gc time)


In [13]:
@time invAiter(A);

  7.872964 seconds (112.60 k allocations: 12.441 GiB, 10.37% gc time)


In [44]:
@time invAME(A);

  8.461736 seconds (17.76 k allocations: 12.430 GiB, 7.49% gc time)


In [45]:
@time invAME1(A);

  7.687437 seconds (173.09 k allocations: 4.995 GiB, 2.91% gc time)


In [15]:
@time inv(A);

  0.304469 seconds (10 allocations: 8.126 MiB)


In [37]:
a = [1.0 2.0
   3.0 4.0]

2×2 Array{Float64,2}:
 1.0  2.0
 3.0  4.0

In [32]:
v = [5.0,6.0]

2-element Array{Float64,1}:
 5.0
 6.0

In [41]:
a[2,:] = v

2-element view(::Array{Float64,2}, 2, :) with eltype Float64:
 5.0
 6.0

In [42]:
a

2×2 Array{Float64,2}:
 1.0  2.0
 5.0  6.0